# Train Classifier

## Libraries

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2" # or any {'0', '1', '2','3'}
from shutil import rmtree


from tensorflow import keras
from keras import applications
from keras import callbacks

##  Definitions

In [2]:
v_num_of_classes=30 #not needed
v_input_shape=(224, 224, 3)
v_path_dataset='../data/sampledataset'
v_image_size=(299,299)
v_color_mode="rgb" # "rgb", "rgba", or "grayscale"
v_batch_size=16 
v_epochs = 10

v_metrics=['MeanSquaredError','AUC','Precision','Recall','accuracy']
v_workspace='./exp'


## Functions

In [3]:
# Dataset
def load_dataset(root_dataset='../data/sampledataset', image_size=(299,299),color_mode='grayscale', batch_size=16 ):
    datagen = keras.preprocessing.image.ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=True,
        featurewise_std_normalization=False,
        samplewise_std_normalization=True,
        zca_whitening=False,
        zca_epsilon=1e-06,
        rotation_range=0,
        width_shift_range=0.0,
        height_shift_range=0.0,
        brightness_range=None,
        shear_range=0.0,
        zoom_range=(0.95,0.95),#####
        channel_shift_range=0.0,
        fill_mode='nearest',
        cval=0.0,
        horizontal_flip=False,
        vertical_flip=False,
        rescale=1./255,#######
        preprocessing_function=None,
        data_format=None,# 'channels_last'
        validation_split=0.2,
        dtype=None,#tf.float32
    )

    ds_train = datagen.flow_from_directory(
        directory=os.path.join (root_dataset),
        target_size=image_size,
        color_mode=color_mode, #"rgb",#grayscale', #
        classes=None,# 'sparse' for integer based instead of hot-encode
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=101,
        save_to_dir=None,
        save_prefix='',
        save_format='png',
        follow_links=False,
        subset='training',
        interpolation='nearest'
    )

    ds_validate = datagen.flow_from_directory(
        directory=os.path.join (root_dataset),
        target_size=image_size,
        color_mode=color_mode, #"rgb",#grayscale', #
        classes=None,# 'sparse' for integer based instead of hot-encode
        class_mode='categorical',
        batch_size=batch_size,
        shuffle=True,
        seed=101,
        save_to_dir=None,
        save_prefix='',
        save_format='png',
        follow_links=False,
        subset='validation',
        interpolation='nearest',
    )

    #class_names=ds_train.class_indices
    #print (class_names)
    #class_names = list(class_names.keys())
    return (ds_train, ds_train.labels),(ds_validate,ds_validate.labels)

In [4]:
rmtree(v_workspace) if os.path.isdir(v_workspace) else None
os.mkdir(v_workspace) 

#os.mkdir(v_workspace) if not(os.path.isdir(v_workspace)) else rmtree(v_workspace)

path_csvname = os.path.join(v_workspace,'performance.csv')
path_tensorboardLog= os.path.join(v_workspace,'tensorboard.csv')
path_checkpoint=os.path.join(v_workspace,'checkpoint')

                            
CSVLogger_cb = callbacks.CSVLogger(path_csvname,  separator=',',   append=True)


TensorBoard_cb = callbacks.TensorBoard(log_dir=path_tensorboardLog,
                                          histogram_freq=1,
                                          write_graph=True,
                                          write_images=True,
                                          update_freq=v_batch_size,
                                          #write_steps_per_second=False,
                                          profile_batch=2,
                                          embeddings_metadata=None)


ModelCheckpoint_cb = callbacks.ModelCheckpoint(filepath=path_checkpoint,
                                                save_best_only=True, ###to save space
                                                save_weights_only=False,
                                                monitor='val_accuracy',
                                                mode='max')


EarlyStopping_cb = callbacks.EarlyStopping(monitor='val_loss', 
                                                   min_delta=0.001,
                                                   patience=4,
                                                   verbose=0, 
                                                   mode='auto',
                                                   baseline=None,
                                                   restore_best_weights=False)

TerminateOnNaN_cb = callbacks.TerminateOnNaN()

ReduceLROnPlateau_cb = callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                            factor=0.01,
                                                            patience=3,
                                                            verbose=0,
                                                            mode='auto',
                                                            min_delta=0.001,
                                                            cooldown=0,
                                                            min_lr=0)

def scheduler(epoch, lr):
    if epoch < 15:
        return lr
    else:
        return lr * tf.math.exp(-0.0001)

LearningRateScheduler_cb = callbacks.LearningRateScheduler(scheduler, 
                                                                    verbose=0)



RemoteMonitor_cb = callbacks.RemoteMonitor(root='http://localhost:9000',
                                                    path='/publish/epoch/end/',
                                                    field='data',
                                                    headers=None,
                                                    send_as_json=False)

## Modeling

In [5]:
# create the base pre-trained model
'''
applications.mobilenet_v2.MobileNetV2
applications.inception_v3.InceptionV3
applications.vgg16.VGG16
applications.xception.Xception

'''

base_model = applications.inception_v3.InceptionV3(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=v_input_shape,
    include_top=False) 

base_model.trainable = False # freeze the base model.

In [6]:
# Create a new model on top.
inputs = keras.Input(shape=v_input_shape)
x = base_model(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(v_num_of_classes)(x)
outputs= keras.layers.Softmax()(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 30)                61470     
                                                                 
 softmax (Softmax)           (None, 30)                0         
                                                                 
Total params: 21,864,254
Trainable params: 61,470
Non-trainable params: 21,802,784
____________________________________________

## Data

In [7]:
(x_train, y_train), (x_valid, y_valid)=load_dataset(root_dataset=v_path_dataset, 
                                                    image_size=v_image_size,
                                                    color_mode=v_color_mode, 
                                                    batch_size=v_batch_size )
num_of_classes=x_train.num_classes
class_indices=x_train.class_indices

Found 680 images belonging to 30 classes.
Found 156 images belonging to 30 classes.


## Training

In [8]:
# compile
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=v_metrics
              )

In [ ]:
#Train the top layer
callbacksList=[CSVLogger_cb,TensorBoard_cb,ModelCheckpoint_cb,EarlyStopping_cb,TerminateOnNaN_cb,ReduceLROnPlateau_cb,LearningRateScheduler_cb]
history=model.fit(x=x_train,
                  validation_data=(x_valid),
                  batch_size=v_batch_size,
                  epochs=v_epochs, 
                  callbacks=callbacksList,
                  shuffle=True
                 )


Epoch 1/10
43/43 [==============================] - 87s 2s/step - loss: 3.0863 - mean_squared_error: 0.0306 - auc: 0.7107 - precision: 0.5000 - recall: 0.0029 - accuracy: 0.2044 - val_loss: 2.4022 - val_mean_squared_error: 0.0274 - val_auc: 0.8783 - val_precision: 1.0000 - val_recall: 0.0256 - val_accuracy: 0.3526 - lr: 0.0010
Epoch 2/10
43/43 [==============================] - 82s 2s/step - loss: 1.7075 - mean_squared_error: 0.0217 - auc: 0.9558 - precision: 0.9231 - recall: 0.1412 - accuracy: 0.5735 - val_loss: 1.9598 - val_mean_squared_error: 0.0238 - val_auc: 0.9205 - val_precision: 0.7826 - val_recall: 0.1154 - val_accuracy: 0.4744 - lr: 0.0010
Epoch 3/10
43/43 [==============================] - ETA: 0s - loss: 1.1447 - mean_squared_error: 0.0150 - auc: 0.9826 - precision: 0.9680 - recall: 0.4000 - accuracy: 0.7500

## Save

In [ ]:
fname = os.path.join (v_workspace,"saved_model")
savedModel = os.path.join(v_workspace, "saved_model")
savedWeights = os.path.join(v_workspace,"saved_weights","weights")

model.save(savedModel)
model.save_weights(savedWeights)